In [20]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
import numpy as np


**Load data && Process**

In [ ]:
X_train = np.load("../data/regression/X_train.npy")
X_test = np.load("../data/regression/X_test.npy")
y_train = np.load("../data/regression/y_train.npy")
y_test = np.load("../data/regression/y_test.npy")

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")

# Split into train/val
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

Train set: (200, 200), Test set: (200, 200)


**Model 1: Ridge Regression**

In [ ]:
ridge_params = {"alpha": [0.001, 0.1, 1.0, 10.0, 100.0, 1000.0]}
ridge = Ridge(random_state=42)
ridge_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(random_state=42))
])
ridge_grid = GridSearchCV(ridge, ridge_params, cv=5, scoring="r2")
ridge_grid.fit(X_tr, y_tr)

best_ridge = ridge_grid.best_estimator_
y_val_pred_ridge = best_ridge.predict(X_val)
print("Ridge best params:", ridge_grid.best_params_)
print("Ridge val R2:", r2_score(y_val, y_val_pred_ridge))

Ridge best params: {'alpha': 1.0}
Ridge val R2: 0.7834015766238961


**Model 2: Lasso Regression**

In [23]:
lasso_params = {"lasso__alpha": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}
lasso_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", Lasso(random_state=42, max_iter=10000))
])

lasso_grid = GridSearchCV(lasso_pipe, lasso_params, cv=5, scoring="r2")
lasso_grid.fit(X_tr, y_tr)

best_lasso = lasso_grid.best_estimator_
y_val_pred_lasso = best_lasso.predict(X_val)
print("Lasso best params:", lasso_grid.best_params_)
print("Lasso val R2:", r2_score(y_val, y_val_pred_lasso))

Lasso best params: {'lasso__alpha': 0.01}
Lasso val R2: 0.918980777367542


**# Model 3: MLP Regressor**

In [24]:
mlp_params = {
    "hidden_layer_sizes": np.array([(50,), (100,), (100, 50)], dtype=object),
    "learning_rate_init": [0.001, 0.01],
    "max_iter": [1000]
}
mlp = MLPRegressor(random_state=42)
mlp_grid = GridSearchCV(mlp, mlp_params, cv=3, scoring="r2")
mlp_grid.fit(X_tr, y_tr.ravel())

best_mlp = mlp_grid.best_estimator_
y_val_pred_mlp = best_mlp.predict(X_val)
print("MLP best params:", mlp_grid.best_params_)
print("MLP val R2:", r2_score(y_val, y_val_pred_mlp))

C:\Users\kenti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:412: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(param_list)


MLP best params: {'hidden_layer_sizes': (100, 50), 'learning_rate_init': 0.01, 'max_iter': 1000}
MLP val R2: 0.7805044915797925


**Final Evaluation (TEST set ONCE)**

In [26]:
print("---- Final Results ----\n")
y_pred_ridge = best_ridge.predict(X_test)
y_pred_lasso = best_lasso.predict(X_test)
y_pred_mlp = best_mlp.predict(X_test)

print("Ridge Test R2:", r2_score(y_test, y_pred_ridge))
print("Lasso Test R2:", r2_score(y_test, y_pred_lasso))
print("MLP Test R2:", r2_score(y_test, y_pred_mlp))

---- Final Results ----

Ridge Test R2: 0.7309094860406974
Lasso Test R2: 0.9344938037526456
MLP Test R2: 0.6690551050866925
